# 0. Load the data - training set and test set

In [56]:
import pandas as pd
df_train = pd.read_csv('./data/train.csv')
df_test = pd.read_csv('./data/test.csv')
df_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [57]:
df_train['Fare'][df_train['Pclass']==1].mean(), df_train['Fare'][df_train['Pclass']==2].mean(), df_train['Fare'][df_train['Pclass']==3].mean()

(84.15468749999992, 20.66218315217391, 13.675550101832997)

In [58]:
df_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [59]:
df_test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

# 1. Data cleaning - fill missing values in "Age", "Cabin" and "Embarked"

### 1.1 For age, we know the basic principle "lady and child first", so we do believe that age, sex and survival have inner relationship.

In **training set**, we group training data by (Sex, Survived), and fill missing values of age by bootstrapping from the corresponding group of (Sex, Survived)

In [60]:
import random

age = {
    'male': {
        1: df_train['Age'][(df_train['Survived']==1) & (df_train['Sex']=='male') & ~df_train['Age'].isnull()].values,
        0: df_train['Age'][(df_train['Survived']==0) & (df_train['Sex']=='male') & ~df_train['Age'].isnull()].values
    },
    'female': {
        1: df_train['Age'][(df_train['Survived']==1) & (df_train['Sex']=='female') & ~df_train['Age'].isnull()].values,
        0: df_train['Age'][(df_train['Survived']==0) & (df_train['Sex']=='female') & ~df_train['Age'].isnull()].values
    }
}

for i in df_train['Age'][df_train['Age'].isnull()].index:
    df_train['Age'][i] = random.choice(age[df_train['Sex'][i]][df_train['Survived'][i]])
df_train['Age'].isnull().sum()

/tmp/ipykernel_266140/3916825549.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



0

In **test set**, we group test data by sex, and fill missing values of age with the mean value from the same sex group in **training set**

In [61]:
mean_age = {
    sex: df_train['Age'][df_train['Sex']==sex].mean() for sex in df_train['Sex'].unique()
}
for i, v in df_test['Age'][df_test['Age'].isnull()].items():
    df_test['Age'][i] = mean_age[df_test['Sex'][i]]
    
df_test['Age'].isnull().sum()

/tmp/ipykernel_266140/1766902384.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



0

### 1.2 For cabin, because most values are missed, we decide to use -1 to fill all missed values for both **training set** and **test set**

In **training set**

In [ ]:
df_train.fillna({'Cabin': -1}, inplace=True)
df_train['Cabin'].isnull().sum()

0

In **test set**

In [ ]:
df_test.fillna({'Cabin': -1}, inplace=True)
df_test['Cabin'].isnull().sum()

0

### 1.3 For embarked, only two values are missed so using bootstrap is very straightforward and effective

In **training set**, we fill missing values by bootstrapping the **training set**

In [ ]:
embarked = df_train['Embarked'][~df_train['Embarked'].isnull()].values
for i in df_train['Embarked'][df_train['Embarked'].isnull()].index:
    df_train['Embarked'][i] = random.choice(embarked)
df_train['Embarked'].isnull().sum()

/tmp/ipykernel_266140/503249784.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



0

In **test set**, we also fill missing values by bootstrapping the **training set**

In [ ]:
for i in df_test['Embarked'][df_test['Embarked'].isnull()].index:
    df_test['Embarked'][i] = random.choice(embarked)
df_test['Embarked'].isnull().sum()

0

### 1.4 Extraly fill the missing *Fare* value in test set

In [ ]:
for i in df_test['Fare'][df_test['Fare'].isnull()].index:
    df_test['Fare'][i] = df_test['Fare'][df_test['Pclass']==df_test['Pclass'][i]].mean()

/tmp/ipykernel_266140/1711049601.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### 1.5 Check if all missing values are filled

In [ ]:
df_train.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [ ]:
df_test.isnull().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

# 2. Separate features and targets

In [ ]:
df_train_features, y_train = df_train.drop(['PassengerId', 'Survived', 'Name', 'Ticket'], axis=1), df_train['Survived'].values
df_test_features = df_test.drop(['PassengerId', 'Name', 'Ticket'], axis=1)
df_train_features

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,male,22.0,1,0,7.2500,-1,S
1,1,female,38.0,1,0,71.2833,C85,C
2,3,female,26.0,0,0,7.9250,-1,S
3,1,female,35.0,1,0,53.1000,C123,S
4,3,male,35.0,0,0,8.0500,-1,S
...,...,...,...,...,...,...,...,...
886,2,male,27.0,0,0,13.0000,-1,S
887,1,female,19.0,0,0,30.0000,B42,S
888,3,female,2.0,1,2,23.4500,-1,S
889,1,male,26.0,0,0,30.0000,C148,C


In [ ]:
df_test_features

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,male,34.500000,0,0,7.8292,-1,Q
1,3,female,47.000000,1,0,7.0000,-1,S
2,2,male,62.000000,0,0,9.6875,-1,Q
3,3,male,27.000000,0,0,8.6625,-1,S
4,3,female,22.000000,1,1,12.2875,-1,S
...,...,...,...,...,...,...,...,...
413,3,male,30.815719,0,0,8.0500,-1,S
414,1,female,39.000000,0,0,108.9000,C105,C
415,3,male,38.500000,0,0,7.2500,-1,S
416,3,male,30.815719,0,0,8.0500,-1,S


# 3. Data preprossing - numerical feature scaling and categorical feature encoding

### 3.1 Numerical feature scaling: Standardization (also called "Z-Score Normalization")

In **training set**

In [ ]:
import plotly.express as px
import os

fig = px.imshow(df_train_features.corr().round(3), text_auto=True)
fig.show()
if not os.path.exists('./plots'):
    os.mkdir('./plots')
fig.write_image('./plots/train_numerical_corr.png')

In [ ]:
from sklearn.preprocessing import StandardScaler

z_scaler = StandardScaler()
numerical_train = df_train[[f for f in df_train_features.corr().columns]].values
numerical_train = z_scaler.fit_transform(numerical_train)
numerical_train

array([[ 0.82737724, -0.53051776,  0.43279337, -0.47367361, -0.50244517],
       [-1.56610693,  0.57914001,  0.43279337, -0.47367361,  0.78684529],
       [ 0.82737724, -0.25310332, -0.4745452 , -0.47367361, -0.48885426],
       ...,
       [ 0.82737724, -1.91758996,  0.43279337,  2.00893337, -0.17626324],
       [-1.56610693, -0.25310332, -0.4745452 , -0.47367361, -0.04438104],
       [ 0.82737724,  0.16301835, -0.4745452 , -0.47367361, -0.49237783]])

In **test set**

In [ ]:
fig = px.imshow(df_test_features.corr().round(3), text_auto=True)
fig.show()
if not os.path.exists('./plots'):
    os.mkdir('./plots')
fig.write_image('./plots/test_numerical_corr.png')

In [ ]:
z_scaler = StandardScaler()
numerical_test = df_test[[f for f in df_test_features.corr().columns]].values
numerical_test = z_scaler.fit_transform(numerical_test)
numerical_test

array([[ 0.87348191,  0.34129386, -0.49947002, -0.4002477 , -0.49731093],
       [ 0.87348191,  1.33030075,  0.61699237, -0.4002477 , -0.5121751 ],
       [-0.31581919,  2.51710902, -0.49947002, -0.4002477 , -0.46399919],
       ...,
       [ 0.87348191,  0.65777607, -0.49947002, -0.4002477 , -0.50769362],
       [ 0.87348191,  0.04979153, -0.49947002, -0.4002477 , -0.49335289],
       [ 0.87348191,  0.04979153,  0.61699237,  0.61989583, -0.23686347]])

# 3.2 Categorical feature encoding - use OneHotEncoder
For Cabin, we only care the firsy letter of Cabin information

In **training set**

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
for i, v in df_train_features['Cabin'].items():
    if isinstance(v, str):
        df_train_features['Cabin'][i] = v[0]
    else:
        df_train_features['Cabin'][i] = str(v)
categorical_train = df_train_features[['Sex', 'Cabin', 'Embarked']].values
categorical_train_encoding = encoder.fit_transform(categorical_train).toarray()
categorical_train_encoding

/tmp/ipykernel_266140/2286952518.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_266140/2286952518.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



array([[0., 1., 1., ..., 0., 0., 1.],
       [1., 0., 0., ..., 1., 0., 0.],
       [1., 0., 1., ..., 0., 0., 1.],
       ...,
       [1., 0., 1., ..., 0., 0., 1.],
       [0., 1., 0., ..., 1., 0., 0.],
       [0., 1., 1., ..., 0., 1., 0.]])

**In test set**

In [ ]:
for i, v in df_test_features['Cabin'].items():
    if isinstance(v, str):
        df_test_features['Cabin'][i] = v[0]
    else:
        df_test_features['Cabin'][i] = str(v)
categorical_test = df_test_features[['Sex', 'Cabin', 'Embarked']].values
categorical_test_encoding = encoder.transform(categorical_test).toarray()
categorical_test_encoding

/tmp/ipykernel_266140/1277409264.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_266140/1277409264.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



array([[0., 1., 1., ..., 0., 1., 0.],
       [1., 0., 1., ..., 0., 0., 1.],
       [0., 1., 1., ..., 0., 1., 0.],
       ...,
       [0., 1., 1., ..., 0., 0., 1.],
       [0., 1., 1., ..., 0., 0., 1.],
       [0., 1., 1., ..., 1., 0., 0.]])

# 4. Construct and concatenate features

In [ ]:
import numpy as np

X_train = np.concatenate([numerical_train, categorical_train_encoding], axis=1)
X_test = np.concatenate([numerical_test, categorical_test_encoding], axis=1)

# 5. Model training and prediction output

In [ ]:
model = {}
y_predict = {}

def train_and_predict(model_name, classifier):
    model[model_name] = classifier
    model[model_name].fit(X_train, y_train)
    y_predict[model_name] = model[model_name].predict(X_test)
    df_test['Survived'] = y_predict[model_name]
    output = df_test[['PassengerId', 'Survived']]
    if not os.path.exists('./submission'):
        os.mkdir('./submission')
    output.to_csv(f'./submission/{model_name}.csv', index=False, header=True)

### 5.1 Logistic Regression Model

In [ ]:
from sklearn.linear_model import LogisticRegression

train_and_predict(model_name='logistic', classifier=LogisticRegression())

### 5.2 Random Forest Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier

train_and_predict(model_name='randomForest', classifier=RandomForestClassifier())

### 5.3 Xgboost Model

In [ ]:
import xgboost as xgb

train_and_predict(model_name='xgboost', classifier=xgb.XGBClassifier())